In [ ]:
import pandas as pd 
sale_date = pd.read_csv("sales_data.csv", parse_dates=['Date'])
# Group the data by the Category column and calculate the following aggregate statistics for each category:
# Total quantity sold.
# Average price per unit.
# Maximum quantity sold in a single transaction.
# Identify the top-selling product in each category based on the total quantity sold.
# Find the date on which the highest total sales (quantity * price) occurred.
sum_mean_P_Q = (
sale_date.groupby(["Category"]).agg(t_quantity_sold = pd.NamedAgg(column = "Quantity", aggfunc='sum')
                                    ,avg_price = pd.NamedAgg(column = "Price", aggfunc= 'mean')

)) 
print(sum_mean_P_Q)
grouped  = sale_date.groupby(["Category", "Product"])["Quantity"].agg("sum").reset_index()
top_products = grouped.loc[grouped.groupby('Category')['Quantity'].idxmax()]
print(top_products)

sale_date["total_sales"] =  sale_date.Quantity*sale_date.Price
top_sales_date = sale_date.groupby("Date")["total_sales"].agg('max')
print(top_sales_date)

In [ ]:
# Group the data by CustomerID and filter out customers who have made less than 20 orders.
import pandas as pd 
customer_orders = pd.read_csv("customer_orders.csv")
f_s_o = customer_orders.groupby("CustomerID")["Quantity"].agg("sum").reset_index()
f_s_o = f_s_o[f_s_o["Quantity"] > 20]
print(f_s_o)

# Identify customers who have ordered products with an average price per unit greater than $120.
customer_orders
avg_price = customer_orders.groupby(["Product","Price"]).mean().reset_index()
rich_customers =  avg_price[avg_price["Price"] > 120]
print(rich_customers)

# Find the total quantity and total price for each product ordered, and filter out products that have a total quantity less than 5 units.
total_P_Q = customer_orders.groupby("Product").agg(total_quantity = pd.NamedAgg(column="Quantity", aggfunc="sum"), 
                                                   total_price = pd.NamedAgg(column="Price", aggfunc= "sum"))
filter = total_P_Q.total_quantity.lt(5)
print(total_P_Q[filter])


In [ ]:
# Homework Assignment 3: Population Salary Analysis

# "task\population.db" sqlite database has population table.
# "task\population salary analysis.xlsx" file defines Salary Band categories.
# Read the data from population table and calculate following measures:
# Percentage of population for each salary category;
# Average salary in each salary category;
# Median salary in each salary category;
# Number of population in each salary category;
# Calculate the same measures in each State
# Note: Use SQL only to select data from database. All the other calculations should be done in python.

In [7]:
import sqlite3 
import pandas as pd
import numpy as np
connection = sqlite3.connect('population.db')
#cursor = connection.cursor()

sql_query = "select * from population"

df = pd.read_sql_query(sql_query, connection, index_col="id")


intervals =  [-np.inf, 200000,400000, 600000,800000, 1000000,1200000,1400000,1600000,1800000, np.inf]
labelss =  [
   ' till $200,000',
'$200,001 - $400,000',
'$400,001 - $600,000',
'$600,001 - $800,000',
'$800,001 - $1,000,000',
'$1,000,001 - $1,200,000',
'$1,200,001 - $1,400,000',
'$1,400,001 - $1,600,000',
'$1,600,001 - $1,800,000',
'$1,800,001 and over' 

]
total_rows = len(df.salary)
df["salary_category"] = pd.cut(df.salary, bins= intervals, labels=labelss )
datea  = df.groupby("salary_category").salary.agg(["mean", "median", "count"]).reset_index()
persentage = df.groupby("salary_category").agg(percentage = pd.NamedAgg(column="salary", aggfunc="count"))/total_rows*100
datea.merge(persentage, on= "salary_category" )



C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1868\4014095227.py:28: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  datea  = df.groupby("salary_category").salary.agg(["mean", "median", "count"]).reset_index()
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1868\4014095227.py:29: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  persentage = df.groupby("salary_category").agg(percentage = pd.NamedAgg(column="salary", aggfunc="count"))/total_rows*100


,salary_category,mean,median,count,percentage
0,"till $200,000",9.928399e+04,98800.0,1151,9.878980
1,"$200,001 - $400,000",2.995581e+05,299882.0,1170,10.042056
2,"$400,001 - $600,000",4.991640e+05,497925.5,1234,10.591366
3,"$600,001 - $800,000",6.996809e+05,701317.0,1156,9.921895
4,"$800,001 - $1,000,000",9.011523e+05,899845.0,1175,10.084971
5,"$1,000,001 - $1,200,000",1.098524e+06,1097765.0,1227,10.531285
6,"$1,200,001 - $1,400,000",1.300685e+06,1300430.0,1131,9.707321
7,"$1,400,001 - $1,600,000",1.499606e+06,1500623.0,1132,9.715904
8,"$1,600,001 - $1,800,000",1.698519e+06,1697481.5,1120,9.612909
9,"$1,800,001 and over",1.902892e+06,1906451.0,1155,9.913312


In [ ]:
# Calculate the same measures in each State

In [9]:
total_rows = len(df.salary)
datea  = df.groupby("state").salary.agg(["mean", "median", "count"]).reset_index()
persentage = round(df.groupby("state").agg(percentage = pd.NamedAgg(column="salary", aggfunc="count"))/total_rows*100, 2)
datea.merge(persentage, on= "state" )

,state,mean,median,count,percentage
0,Alabama,9.750741e+05,986050.5,246,2.11
1,Alaska,1.004373e+06,970124.0,54,0.46
2,Arizona,1.044695e+06,1096324.0,253,2.17
3,Arkansas,1.130035e+06,1136530.0,60,0.51
4,California,9.989894e+05,1017228.0,1332,11.43
5,Colorado,1.030555e+06,987728.0,287,2.46
6,Connecticut,1.004350e+06,1025583.0,182,1.56
7,Delaware,9.220878e+05,987908.0,35,0.30
8,District of Columbia,1.002837e+06,961365.5,370,3.18
9,Florida,9.944462e+05,972755.5,924,7.93
